In [ ]:
import cml.data_v1 as cmldata

import configparser
import uuid
import os
from typing import Dict
from pyspark.sql.functions import to_date, col
from pyspark import SparkConf
from pyspark.sql import SparkSession, DataFrameWriter
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta, date
from pyspark.sql.functions import year, month, count, sum, col, ceil, when
import pandas as pd

# Sample in-code customization of spark configurations
#from pyspark import SparkContext
#SparkContext.setSystemProperty('spark.executor.cores', '1')
#SparkContext.setSystemProperty('spark.executor.memory', '2g')

CONNECTION_NAME = "pdnd-prod-dl-1"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()

In [ ]:
df_gold = spark.sql( """   
           SELECT  p.iun,
           p.requestid,
           p.affido_recapitista_con016_data,
           CASE 
               WHEN n.tms_viewed IS NULL THEN n.tms_effective_date
               WHEN n.tms_effective_date IS NULL THEN n.tms_viewed
               WHEN n.tms_viewed < n.tms_effective_date THEN n.tms_viewed
               ELSE n.tms_effective_date
           END AS tms_perfezionamento
   FROM send.gold_notification_analytics n JOIN send.gold_postalizzazione_analytics p ON (n.iun = p.iun)
   WHERE p.affido_recapitista_con016_data IS NOT NULL AND p.senderpaid = "53b40136-65f2-424b-acfb-7fae17e35c60" AND ( n.tms_viewed IS NOT NULL OR n.tms_effective_date IS NOT NULL )"""   
                    ) 

In [ ]:
df_gold.createOrReplaceTempView("DF_GOLD")

In [ ]:
df_postalizzazione_ranked = spark.sql( """ 
SELECT iun, requestid, pcretry_rank, attempt_rank
FROM (
	SELECT iun,
		   requestid,
		   pcretry_rank,
		   ROW_NUMBER() OVER (PARTITION BY iun ORDER BY attempt_number DESC) AS attempt_rank
	FROM send.gold_postalizzazione_analytics
	WHERE scarto_consolidatore_stato IS NULL AND senderpaid = "53b40136-65f2-424b-acfb-7fae17e35c60"
) RankedAttempts
WHERE pcretry_rank = 1 AND attempt_rank = 1;
""")

In [ ]:
df_postalizzazione_ranked.createOrReplaceTempView("DF_RANKED")

In [ ]:
df_gold_ranked = spark.sql(""" 
    SELECT dg.*
    FROM DF_RANKED dr JOIN DF_GOLD dg ON (dr.requestid = dg.requestid)
""")

In [ ]:
#print(df_gold_notification)
df_gold_ranked = df_gold_ranked.withColumn("affido_recapitista_con016_data", F.col("affido_recapitista_con016_data").cast("timestamp"))
df_gold_ranked = df_gold_ranked.withColumn("tms_perfezionamento", F.col("tms_perfezionamento").cast("timestamp"))

In [ ]:
#Calcolo delle tempistiche - aggiunta colonna
df_gold_ranked = df_gold_ranked.withColumn(
                        "diff_affido_perfezionamento",
                        F.round((F.unix_timestamp("tms_perfezionamento") - F.unix_timestamp("affido_recapitista_con016_data")) / (3600 * 24),2)
                    )

In [ ]:
#Calcolo delle perfezionate totali 
total_notifications = df_gold_ranked.count()

total_notifications

In [ ]:
#Calcolo dei tempi medi di perfezionamento raggruppati per mese/anno di deposito sentat?

#dic 24 - somma tempistiche / notifiche perfezionate dicembre
#gen 25 - somma tempistiche / notifiche perfezionate gennaio
#feb 25 - somma tempistiche / notifiche perfezionate febbraio

df_grouped = df_gold_ranked.withColumn("anno_affido", F.year(F.col("affido_recapitista_con016_data"))) \
                    .withColumn("mese_affido", F.month(F.col("affido_recapitista_con016_data")))



In [ ]:
print(df_grouped)

In [ ]:
df_tempo_medio = df_grouped.groupBy("anno_affido", "mese_affido").agg(
    sum("diff_affido_perfezionamento").alias("somma_tempistiche"),
    count("tms_perfezionamento").alias("notifiche_perfezionate")
)

In [ ]:
df_tempo_medio = df_tempo_medio.withColumn(
    "tempo_medio_perfezionamento", F.round(col("somma_tempistiche") / col("notifiche_perfezionate"), 2)
)

In [ ]:
print(df_tempo_medio)

# Esportare il risultato in tabella

In [ ]:
df_tempo_medio.createOrReplaceTempView("DF_OUTPUT")

In [ ]:
 spark.sql("""SELECT * FROM DF_OUTPUT""").writeTo("send_dev.inps_affido_perfezionamento")\
                .using("iceberg")\
                .tableProperty("format-version","2")\
                .tableProperty("engine.hive.enabled","true")\
                .createOrReplace()
#print(datetime.now()-start)